# Semester Project : Information Retrieval System
 ### Submitted to Ma'am Naima
- #### Submitted by : Hassan Ashiq BESE 23 C , Usman Ali Abbasi 23 C
- ###### Link to my GitHub Repository <a href="https://github.com/hassanashiqasse/PCA">Click Here</a>

### Importing Libraries

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

### Reading CISI.ALL File

In [2]:
with open('IR Project/CISI.ALL') as CISI_file:
    lines = ""
    for l in CISI_file.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
print("Done")

Done


### Placing each document in CISI File in a dictionary

In [3]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")


Number of documents = 1460.



### Reading CISI Query File and placing each query in a dictionary

In [4]:
with open('IR Project/datasets.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
    
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print("Query # 2 : ", qry_set["2"]) # note that the dictionary indexes are strings, not numbers. 

Number of queries = 112.

Query # 2 :  How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?


### Reading CISI Ground Truth File and placing each in a dictionary

In [5]:
rel_set = {}
with open('IR Project/datasets.REL') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)
    
# Print something to see the dictionary structure, etc.
print(f"\nNumber of mappings = {len(rel_set)}" + ".\n")
print("Result for Rel#7 :" , rel_set["7"]) # note that the dictionary indexes are strings, not numbers. 


Number of mappings = 76.

Result for Rel#7 : ['194', '263', '374', '442', '668', '835', '1180', '1428']


### Pre Process of Data

In [6]:
def convert_lower_case(data):
    return np.char.lower(data)

In [7]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [8]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [9]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [10]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [11]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [12]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    #data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    #data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [13]:
processed_set={}
proc_token_id=""
proc_token_text=""

for i in doc_set:
    doc_token_id=i
    processed_set[doc_token_id]=preprocess(doc_set[str(i)])
print("done")
    

done


In [14]:
processed_set["2"]

' use made technic librari slater report analysi 6300 act use 104 technic librari unit kingdom librari use one aspect wider pattern inform use inform transfer librari restrict use document take account document use outsid librari still less inform transfer oral person person librari act channel proport situat inform transfer take technic inform transfer whole doubt proport major one user technic inform particularli technolog rather scienc visit librari rare reli desk collect handbook current period person contact colleagu peopl organ even regular librari user also receiv inform way'

#### Converting prcessed text to tokens and placing in a dictionary where keys are the docs id

In [15]:
tokens_set={}
doc_token_id=""
doct_token_text=""

for i in processed_set:
    doc_token_id=i
    tokens_set[doc_token_id]=word_tokenize(processed_set[str(i)])
print("done")
    

done


#### Calculating DF

In [17]:
DF = {}

for i in range(len(tokens_set)):
    tokens = tokens_set[str(i+1)]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [18]:
DF

{'18': 9,
 'edit': 44,
 'dewey': 13,
 'decim': 16,
 'classif': 105,
 'comaromi': 1,
 'present': 318,
 'studi': 362,
 'histori': 52,
 'first': 175,
 'ddc': 5,
 'publish': 122,
 '1876': 4,
 'eighteenth': 1,
 '1971': 27,
 'futur': 95,
 'continu': 68,
 'appear': 86,
 'need': 251,
 'spite': 7,
 'long': 48,
 'healthi': 1,
 'life': 39,
 'howev': 111,
 'full': 41,
 'stori': 4,
 'never': 15,
 'told': 4,
 'biographi': 3,
 'briefli': 34,
 'describ': 271,
 'system': 515,
 'attempt': 125,
 'provid': 251,
 'detail': 101,
 'work': 252,
 'spur': 3,
 'growth': 67,
 'librarianship': 49,
 'countri': 46,
 'abroad': 5,
 'use': 659,
 'made': 211,
 'technic': 130,
 'librari': 555,
 'slater': 5,
 'report': 185,
 'analysi': 226,
 '6300': 1,
 'act': 27,
 '104': 1,
 'unit': 94,
 'kingdom': 10,
 'one': 341,
 'aspect': 103,
 'wider': 17,
 'pattern': 85,
 'inform': 660,
 'transfer': 30,
 'restrict': 25,
 'document': 251,
 'take': 66,
 'account': 66,
 'outsid': 19,
 'still': 32,
 'less': 66,
 'oral': 4,
 'person': 6

In [19]:

total_vocab_size = len(DF)


In [20]:
total_vocab = [x for x in DF]
N=len(total_vocab)
N

7132

In [21]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [22]:
doc = 0
N=len(tokens_set)
tf_idf = {}

for i in range(len(tokens_set)):
    if(i>0):
        tokens = tokens_set[str(i)]
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc,token] = tf*idf
    doc += 1

print("tf-idf done")

tf-idf done


In [23]:
def matching_score(query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    

print("Matching scores done")

Matching scores done


In [24]:
matching_score(qry_set["3"])

Matching Score

Query: What is information science?  Give definitions where possible.

['inform', 'scienc', 'give', 'definit', 'possibl']

[469, 1181, 1179, 1133, 1142, 599, 60, 372, 540, 1087]


## cosine similarilty

In [25]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [26]:
D = np.zeros((N, total_vocab_size))   #total_vocab_size is the length of DF
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [27]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [29]:
def cosine_similarity(k, query):
    
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    #print("\nQuery:", query)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    
    #print("Most similar Dpocuments-IDs : ")
    
    #print(out)
    
    return out


In [30]:
Q = cosine_similarity(20,qry_set["3"])
Q

array([ 469,  445, 1181, 1179,  540, 1142, 1133,   85, 1235, 1077,  599,
        803,  914, 1030,  177,  456,  553,   42,  372, 1137], dtype=int64)

### Reading ground truth


In [31]:
rel_set = {}
with open('IR Project/Datasets.REL') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id) 
    
    
print(rel_set["3"]) # note that the dictionary indexes are strings, not numbers. 
    

[60, 85, 118, 123, 126, 131, 133, 137, 138, 160, 346, 359, 363, 372, 412, 445, 454, 461, 463, 469, 532, 539, 540, 553, 554, 555, 585, 590, 599, 640, 660, 664, 803, 901, 909, 911, 1030, 1053, 1077, 1179, 1181, 1190, 1191, 1326]


### Precision and recall

In [32]:
precision_list=[]
recall_list=[]

In [33]:
precision_list=[]
recall_list=[]

for i in range(1,len(doc_set)):
    try:
        result_from_cosine=cosine_similarity(6 , qry_set[str(i)]).tolist()
        result_from_ground_truth=rel_set[str(i)]
        
        true_Positive=len(set(result_from_cosine) & set(result_from_ground_truth)) #set(a) & set(b) gives us intersection between a and b
        false_Positive=len(np.setdiff1d(result_from_cosine , result_from_ground_truth))
        false_Negative=len(np.setdiff1d(result_from_ground_truth , result_from_cosine))
        #print("true psotive",true_Positive)
        #print("false negative",false_Negative)
        
        try:
            precission= (true_Positive) / ( true_Positive + false_Positive )
            recall= (true_Positive) / (true_Positive + false_Negative)
        except ZeroDivisionError:
            pass

        precision_list.append(precission)
        recall_list.append(recall)
        
        
        
    except KeyError:
        pass
    

In [34]:
sum(precision_list)

35.99999999999999

In [35]:
sum(recall_list)

11.822298142574425

## Enter a Query, Get your Result 
### Simple User Interface

In [64]:
query=input("Enter your query here : ")

Q=cosine_similarity(10,query)

print("\n\nEntered Query is : " , query)
print("\n\nRelated Documents IDs are : ", Q)
print("\nDo you want to retrive the document ? \n press Y to see all related docs \n Press S to see a single document with given id \n Press N to exit ")

entered_option=input()
    
if entered_option == "Y":

    print("\n\n*** You are in All Document Retriveal Mood ***\n\n")

    for i in range(len(Q)):
            print("\n\nDoc-Id :", Q[i] , "\n\t" ,doc_set[str(Q[i])])
           
elif entered_option == "S":
    print("Enter your desired document ID : ")
    doc_id=input()
    print("Doc-Id : ", doc_id, "\n\t" ,doc_set[doc_id])
        

else:
    print("Thank you for using our Information System")
    print("Hassan Ashiq & Usman Ali Abbasi")

Enter your query here : 'What is information science?  Give definitions where possible.'


Entered Query is :  'What is information science?  Give definitions where possible.'


Related Documents IDs are :  [ 469  445 1181 1179  540 1142 1133   85 1235 1077]

Do you want to retrive the document ? 
 press Y to see all related docs 
 Press S to see a single document with given id 
 Press N to exit 
Y


*** You are in All Document Retriveal Mood ***




Doc-Id : 469 
	 The Phenomena of Interest to Information Science Wersig, Gernot Neveling, Ulrich Discusses the various explicit and implicit definitions of information and information science, against a view of their historical development.. Shows how the various views of information science overlap with other disciplines, and concludes with a proposal for a definition of information science based on social need.. A schema of information sciences is put forward with the plea that any discussion of information and information science should

In [63]:
qry_set["3"]

'What is information science?  Give definitions where possible.'

 ## IR Semester Project
 ### Submitted to Ma'am Naima
- #### Submitted by : Hassan Ashiq BESE 23 C , Usman Ali Abbasi 23 C
- ###### Link to my GitHub Repository <a href="https://github.com/hassanashiqasse/PCA">Click Here</a>